In [100]:
import os
import sys
import re
import pickle
import numpy as np
from pprint import pprint
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import backend
from tensorflow.python.client import device_lib
from tensorflow.keras.models import model_from_json

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#### Uncomment on server
# device = device_lib.list_local_devices()
# print(device)
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# backend.set_session(sess)

In [165]:
#CONSTANTS
VALIDATION_SPLIT = 0.3
TEST_SPLIT = 0.2
learning_rate = .001
max_grad_norm = 1.
DROPOUT = 0.5
EMBEDDING_DIM = 200

In [3]:
# PATH CONSTANTS
PICKLE_ROOT = 'data/lyrics/'
PICKLE_INPUT = 'CNN_input.pickle' 

EMBEDDING_PATH = 'data/glove_embeddings/'
EMBEDDING_FILE = 'glove.6B.'+str(EMBEDDING_DIM)+'d.txt'

MODEL_SAVE_FILE = 'cnn_model_1.0.json'
MODEL_SAVE_WEIGHTS_FILE = 'cnn_model_1.0.h5'

In [160]:
# Default values - changed later
MAX_SONG_LENGTH = 2500
MAX_UNIQUE_WORDS = 20000


In [39]:
# Embedding
# Elmo could improve the word embeddings - need more research
# elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
print('loading embedding')
if not os.path.exists(EMBEDDING_PATH+EMBEDDING_FILE):
    print('Embeddings not found, downloading now')
    print(os.system('pwd'))
    os.system(' cd ' + DATA_PATH)
    os.system(' mkdir ' + EMBEDDING_DIR)
    os.system(' cd ' + EMBEDDING_DIR)
    os.system(' wget http://nlp.stanford.edu/data/glove.6B.zip')
    os.system(' unzip glove.6B.zip')
    os.system(' cd ../..')

glove_embeddings = {}
with open(EMBEDDING_PATH+EMBEDDING_FILE, encoding='utf-8') as emb_f:
    for line in emb_f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_embeddings[word] = vector
print('finished loading embedding')

loading embedding
finished loading embedding


In [191]:

print('loading pickles')
pickle_data = pickle.load( open(PICKLE_ROOT + PICKLE_INPUT , "rb" ))
lyrics = pickle_data['lyrics']
lyrics_labels = pickle_data['lyrics_labels']
unique_words_set = pickle_data['unique_words_set']
genre_index = pickle_data['genre_index']
MAX_SONG_LENGTH = round(pickle_data['longest_song'],-2)
print('number of songs: %d' %(len(lyrics)))
print('number of genres: %d' %(len(genre_index)))
print('number of lyrics: %d' %(len(lyrics_labels)))
print('number of unique words: %d' %(len(unique_words_set)))
print('longest song: %d' %(MAX_SONG_LENGTH))
print('finished loading pickles')

loading pickles
number of songs: 22500
number of genres: 5
number of lyrics: 22500
number of unique words: 54404
longest song: 1300
finished loading pickles


In [192]:
# MAX_UNIQUE_WORDS = len(unique_words_set)
# MAX_SONG_LENGTH = 1000
# data preparing
print('tokenizing')
tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_UNIQUE_WORDS)
tokenizer.fit_on_texts(lyrics)
sequences = tokenizer.texts_to_sequences(lyrics)

word_index = tokenizer.word_index
print('Unique words tokens %d' % (len(word_index)))

data = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SONG_LENGTH,padding='post')
labels = keras.utils.to_categorical(np.asarray(lyrics_labels))

print('finished tokenizing')


tokenizing
Unique words tokens 54391
finished tokenizing


In [152]:
# save model
def save_model(nn_model,filename,weights_filename):
    # serialize model to JSON
    model_json = nn_model.to_json()
    with open(filename, "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    nn_model.save_weights(weights_filename)
    print("Saved model to disk")


In [153]:

def load_model(filename,weights_filename):
    # load json and create model
    json_file = open(filename, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(weights_filename)
    print("Loaded model from disk")
    return loaded_model


In [193]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]



t_data = data[:int(data.shape[0]*.1)]
t_labels = labels[:int(data.shape[0]*.1)]

# NO GPU so must downsize
CPU=False
if not CPU:
    num_test_samples = int(TEST_SPLIT * data.shape[0])
    x_test = data[:num_test_samples]
    y_test = labels[:num_test_samples]
    x_train = data[num_test_samples:]
    y_train = labels[num_test_samples:]
    
else:
    num_test_samples = int(TEST_SPLIT * t_data.shape[0])

    x_test = t_data[:num_test_samples]
    y_test = t_labels[:num_test_samples]
    x_train = t_data[num_test_samples:]
    y_train = t_labels[num_test_samples:]
    
print('data tensor:', data.shape)
print('test tensor:', x_test.shape)
print('train tensor:', x_train.shape)
print('valid splits: ', x_test.shape[0]+x_train.shape[0] == data.shape[0])
print('label tensor:', labels.shape)
print('test tensor:', y_test.shape)
print('train tensor:', y_train.shape)
print('valid splits: ', y_test.shape[0]+y_train.shape[0] == data.shape[0])

print('Preparing embedding matrix.')
# prepare embedding matrix
unique_words_count = min(MAX_UNIQUE_WORDS, len(word_index))
embedding_matrix = np.zeros((unique_words_count, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= MAX_UNIQUE_WORDS:
        continue
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        # potentially can improve if OOV words are handled differently        
        embedding_matrix[i] = embedding_vector

data tensor: (22500, 1300)
test tensor: (450, 1300)
train tensor: (1800, 1300)
valid splits:  False
label tensor: (22500, 5)
test tensor: (450, 5)
train tensor: (1800, 5)
valid splits:  False
Preparing embedding matrix.


In [230]:
print('Building model')
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = keras.layers.Embedding(unique_words_count,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SONG_LENGTH,
                            trainable=False)

CONV1D_OUT = 128
OUT = 5
MAX_POOLING_1 = 5
MAX_POOLING_2 = 47
DROPOUT = 0.5

# conv1d = single spatial convolution of 2d input (sequence of 1000 - 200 demention vectors)
# maxpooling1d = randomly downsizes by pooling val
# dropout = randomly zeros at dropout rate to avoid overfitting

model2 = Sequential()
model2.add(embedding_layer)
model2.add(tf.keras.layers.Conv1D(CONV1D_OUT, OUT, activation='relu'))
model2.add(tf.keras.layers.MaxPooling1D(MAX_POOLING_1))
model2.add(tf.keras.layers.Dropout(DROPOUT))

model2.add(tf.keras.layers.Conv1D(CONV1D_OUT, OUT, activation='relu'))
model2.add(tf.keras.layers.MaxPooling1D(MAX_POOLING_1))
model2.add(tf.keras.layers.Dropout(DROPOUT))

model2.add(tf.keras.layers.Conv1D(CONV1D_OUT, OUT, activation='relu'))
model2.add(tf.keras.layers.MaxPooling1D(MAX_POOLING_2))
model2.add(tf.keras.layers.Flatten())

model2.add(tf.keras.layers.Dense(CONV1D_OUT, activation='relu'))
model2.add(tf.keras.layers.Dense(len(genre_index), activation='softmax'))
# loss=binary_crossentropy
# model2.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['acc'])
# adam = tf.keras.optimizers.Adam(lr=learning_rate, clipnorm=max_grad_norm)
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model2.summary()

Building model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_51 (Embedding)     (None, 1300, 200)         4000000   
_________________________________________________________________
conv1d_163 (Conv1D)          (None, 1296, 128)         128128    
_________________________________________________________________
max_pooling1d_162 (MaxPoolin (None, 259, 128)          0         
_________________________________________________________________
dropout_108 (Dropout)        (None, 259, 128)          0         
_________________________________________________________________
conv1d_164 (Conv1D)          (None, 255, 128)          82048     
_________________________________________________________________
max_pooling1d_163 (MaxPoolin (None, 51, 128)           0         
_________________________________________________________________
dropout_109 (Dropout)        (None, 51, 128)           0     

In [231]:
print('Training Model')
model_details = model2.fit(x_train, y_train,
            epochs=100,
            shuffle=True,
            verbose=1,
            validation_split=VALIDATION_SPLIT)

scores= model2.evaluate(x_test,y_test,verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Training Model
Train on 1260 samples, validate on 540 samples
Epoch 1/5
1260/1260 [==============================] - 56s 44ms/sample - loss: 1.5732 - acc: 0.3278 - val_loss: 1.5471 - val_acc: 0.2130
Epoch 2/5
1260/1260 [==============================] - 46s 36ms/sample - loss: 1.3459 - acc: 0.3913 - val_loss: 1.3521 - val_acc: 0.4593
Epoch 3/5
1260/1260 [==============================] - 44s 35ms/sample - loss: 1.2216 - acc: 0.4611 - val_loss: 1.3190 - val_acc: 0.4167
Epoch 4/5
1260/1260 [==============================] - 45s 36ms/sample - loss: 1.1457 - acc: 0.4905 - val_loss: 1.2038 - val_acc: 0.4778
Epoch 5/5
450/450 [==============================] - 5s 11ms/sample - loss: 1.2185 - acc: 0.4756
Test loss: 1.2184653268920052
Test accuracy: 0.47555557


In [232]:
def check_accuracy(model2,x_test,y_test):
    print('%d,%d'%(len(x_test),len(y_test)))
    correct=0.0
    for x,y in zip(x_test, y_test):
        x = np.reshape(x,(1,-1))
        y = np.reshape(y,(1,-1))
        scores = model2.predict(x,verbose=0)
    #     print(scores)
    #     print(y)
    #     print(np.argmax(scores))
    #     print(np.argmax(y))
        if np.argmax(scores)==np.argmax(y):
            correct+=1.0
    print('Accuracy: %.2f' %(100*correct/len(x_test)))
    #     print('Test loss:', scores[0])
    #     print('Test accuracy:', scores[1])

450,450
[[0.04111464 0.1173034  0.18286471 0.26814574 0.39057153]]
[[1. 0. 0. 0. 0.]]
4
0
[[0.1965251  0.28190386 0.3199247  0.17682219 0.0248241 ]]
[[0. 1. 0. 0. 0.]]
2
1
[[0.5268325  0.12848248 0.16914529 0.15460691 0.02093284]]
[[0. 1. 0. 0. 0.]]
0
1
[[0.32973132 0.14795423 0.23406172 0.19596085 0.09229193]]
[[0. 0. 0. 0. 1.]]
0
4
[[0.11366777 0.17415382 0.24691154 0.41043133 0.05483562]]
[[0. 0. 1. 0. 0.]]
3
2
[[0.0426743  0.09643818 0.13225634 0.24737142 0.48125976]]
[[0. 0. 0. 0. 1.]]
4
4
[[0.15136635 0.21256886 0.2517785  0.32655555 0.05773072]]
[[0. 0. 0. 1. 0.]]
3
3
[[0.2709633  0.17911671 0.23831306 0.26950377 0.04210321]]
[[1. 0. 0. 0. 0.]]
0
0
[[0.3339579  0.1875961  0.22211722 0.2005912  0.05573751]]
[[1. 0. 0. 0. 0.]]
0
0
[[0.35238552 0.17433812 0.205534   0.2283782  0.03936415]]
[[1. 0. 0. 0. 0.]]
0
0
[[0.8435283  0.03726201 0.06452474 0.04977695 0.00490807]]
[[1. 0. 0. 0. 0.]]
0
0
[[0.2919231  0.19993655 0.2558455  0.19353169 0.05876316]]
[[1. 0. 0. 0. 0.]]
0
0
[[0.0726

[[0.42979214 0.13371299 0.18786855 0.2124151  0.03621136]]
[[1. 0. 0. 0. 0.]]
0
0
[[0.35940439 0.18269981 0.22740358 0.19436014 0.03613213]]
[[0. 0. 1. 0. 0.]]
0
2
[[0.20719537 0.21309428 0.25102714 0.25614092 0.07254229]]
[[0. 0. 0. 1. 0.]]
3
3
[[0.19337073 0.20685911 0.26060992 0.2783373  0.06082289]]
[[0. 0. 0. 1. 0.]]
3
3
[[0.4646483  0.13496043 0.21936649 0.15249753 0.0285273 ]]
[[0. 1. 0. 0. 0.]]
0
1
[[0.6685851  0.07574871 0.12466954 0.12177774 0.00921892]]
[[0. 0. 1. 0. 0.]]
0
2
[[0.06801043 0.15780461 0.23542668 0.4931791  0.04557916]]
[[0. 0. 0. 1. 0.]]
3
3
[[0.37746897 0.14849214 0.20833942 0.23386092 0.03183853]]
[[1. 0. 0. 0. 0.]]
0
0
[[0.01861283 0.04070715 0.05346465 0.09664057 0.7905748 ]]
[[0. 0. 0. 0. 1.]]
4
4
[[0.04828028 0.16168715 0.22600676 0.341972   0.22205378]]
[[0. 0. 0. 0. 1.]]
3
4
[[0.11268147 0.24296041 0.26057982 0.31542274 0.06835555]]
[[0. 0. 0. 1. 0.]]
3
3
[[0.1926615  0.19737236 0.25729316 0.30944365 0.04322932]]
[[0. 0. 0. 1. 0.]]
3
3
[[0.31267804 0.1

[[0.15557678 0.23658285 0.26903743 0.27996534 0.05883764]]
[[1. 0. 0. 0. 0.]]
3
0
[[0.06661064 0.36855063 0.32048672 0.21866603 0.02568598]]
[[0. 1. 0. 0. 0.]]
1
1
[[0.0812467  0.27431202 0.28509042 0.30537364 0.05397727]]
[[0. 1. 0. 0. 0.]]
3
1
[[0.16495606 0.23313238 0.29961056 0.24506266 0.0572383 ]]
[[0. 0. 0. 1. 0.]]
2
3
[[0.20388038 0.21100436 0.24396676 0.26804736 0.07310105]]
[[0. 0. 0. 1. 0.]]
3
3
[[0.32228422 0.16969366 0.25158262 0.21114637 0.04529323]]
[[0. 0. 0. 1. 0.]]
0
3
[[0.12211154 0.16958356 0.24764143 0.3560203  0.10464318]]
[[0. 0. 1. 0. 0.]]
3
2
[[0.21430351 0.18473682 0.24020477 0.3206206  0.04013432]]
[[0. 1. 0. 0. 0.]]
3
1
[[0.8446143  0.03713641 0.06418754 0.04881557 0.00524625]]
[[1. 0. 0. 0. 0.]]
0
0
[[0.0881087  0.2473122  0.2987028  0.32454365 0.04133271]]
[[0. 0. 0. 1. 0.]]
3
3
[[0.00462462 0.01878275 0.02448728 0.04526994 0.9068354 ]]
[[0. 0. 0. 0. 1.]]
4
4
[[0.14943965 0.22260144 0.25330827 0.29917988 0.07547073]]
[[0. 1. 0. 0. 0.]]
3
1
[[0.0145943  0.0

[[0.15338539 0.19286971 0.25432742 0.3390302  0.06038727]]
[[0. 0. 1. 0. 0.]]
3
2
[[0.165763   0.26487592 0.28132826 0.22445716 0.06357567]]
[[0. 0. 0. 1. 0.]]
2
3
[[0.13724495 0.25599214 0.28929597 0.27315074 0.04431628]]
[[0. 1. 0. 0. 0.]]
2
1
[[0.09191187 0.19965325 0.2566262  0.37737694 0.07443172]]
[[1. 0. 0. 0. 0.]]
3
0
[[0.04277693 0.18954588 0.22220512 0.2284728  0.31699926]]
[[0. 0. 0. 0. 1.]]
4
4
[[0.1345527  0.17573534 0.25030348 0.3742704  0.06513808]]
[[1. 0. 0. 0. 0.]]
3
0
[[0.33902606 0.21103846 0.27027345 0.14722556 0.0324365 ]]
[[0. 0. 1. 0. 0.]]
0
2
[[0.2077023  0.21225958 0.2504341  0.26043975 0.0691643 ]]
[[0. 0. 0. 1. 0.]]
3
3
[[0.07364971 0.19020872 0.24934654 0.4356981  0.05109692]]
[[0. 1. 0. 0. 0.]]
3
1
[[0.09674973 0.2130285  0.25253057 0.35010168 0.08758944]]
[[0. 0. 1. 0. 0.]]
3
2
[[0.00630127 0.02661024 0.03528767 0.08215077 0.84964997]]
[[0. 0. 0. 0. 1.]]
4
4
[[0.23849402 0.19909532 0.25877357 0.26398602 0.03965104]]
[[0. 1. 0. 0. 0.]]
3
1
[[0.04439897 0.1

[[0.13449858 0.27333364 0.30327725 0.2460455  0.04284498]]
[[0. 0. 0. 1. 0.]]
2
3
[[0.19779156 0.2278843  0.2876573  0.23168164 0.05498521]]
[[0. 0. 0. 0. 1.]]
2
4
[[0.44373518 0.13969284 0.22388367 0.15965532 0.03303302]]
[[0. 0. 1. 0. 0.]]
0
2
[[0.33982876 0.19645642 0.22189051 0.21265015 0.02917416]]
[[0. 1. 0. 0. 0.]]
0
1
[[0.18929195 0.20284338 0.25183487 0.30237767 0.05365217]]
[[0. 0. 0. 1. 0.]]
3
3
[[0.44109073 0.13617934 0.18024535 0.21747534 0.02500921]]
[[0. 0. 0. 1. 0.]]
0
3
[[0.03361477 0.08846494 0.11238787 0.17251195 0.5930205 ]]
[[0. 0. 0. 0. 1.]]
4
4
[[0.55542237 0.11557078 0.15819903 0.14703585 0.02377206]]
[[1. 0. 0. 0. 0.]]
0
0
[[0.10225121 0.16817808 0.25651172 0.42588234 0.04717665]]
[[0. 0. 0. 1. 0.]]
3
3
[[0.2110085  0.23675302 0.2796782  0.21439426 0.05816601]]
[[0. 0. 1. 0. 0.]]
2
2
[[0.01465449 0.03428873 0.04659971 0.10567369 0.7987834 ]]
[[0. 0. 0. 0. 1.]]
4
4
[[0.09664513 0.2312435  0.27740714 0.33767965 0.05702464]]
[[0. 1. 0. 0. 0.]]
3
1
[[0.07196657 0.1

In [ ]:
def plot_data(history):
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
#     plt.show()
    plt.savefig('accuracy.png')
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
#     plt.show()
    plt.savefig('loss.png')


In [ ]:
check_accuracy(model2,x_test,y_test)
plot_data(model_details)
save_model(model2,MODEL_SAVE_FILE, MODEL_SAVE_WEIGHTS_FILE)